# Kafka Consumer

Using the `kafka-python` module, Kafka consumers can be instantiated via the `KafkaConsumer` class:

```python
#--- A TYPICAL CONSUMER
consumer = KafkaConsumer(
    bootstrap_servers=['62.30.10.23:9092'],  #<<<--- list of brokers
    security_protocol="SSL",                 #<<<--- security protocol (if any) 
    ssl_cafile="./ca.pem",                   #<<<--- certificate details (if any)
    ssl_certfile="./service.cert",           #           ...
    ssl_keyfile="./service.key",             #           ...
    value_deserializer=msgpack.unpackb,      #<<<--- message value deserialization function (e.g. unpack the message from a specific format)
    auto_offset_reset='earliest',            #<<<--- automatically bring the reading offset to the earliest message
    group_id="group_A",                      #<<<--- identify this consumer as part of group_A
)
```

Once more, we'll use a simple implementation of the consumer, with no specific configurations used in this example.

In [ ]:
# define the list of brokers in the cluster
KAFKA_BOOTSTRAP_SERVERS = ['kafka-broker:9092']

In [ ]:
from kafka import KafkaConsumer

# create a Kafka consumer instance
consumer = KafkaConsumer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,  # list of Kafka brokers
    consumer_timeout_ms=10000                   # maximum time to wait for a new message 
                                                # before stopping the consumer
)

Inspect the available topics on the brokers:

In [ ]:
# list all available topics on the kafka brokers
consumer.topics()

In the Publish/Subscribe (Pub/Sub) model, before consuming messages from a specific topic, you need to subscribe to that topic. By subscribing, the consumer expresses its interest in receiving messages from the specified topic.

It's important to note that subscribing to a topic doesn't immediately consume any messages. 

Instead, it establishes a connection between the consumer and the partitions of the subscribed topic hosted on the Kafka brokers. This connection allows the consumer to start polling for messages from those partitions.

Once you have subscribed to the topic, the consumer can start polling for messages, fetching messages from the subscribed partitions and return them to the consumer for further processing.

In [ ]:
# subscribe to a topic
consumer.subscribe('my_awesome_topic')

# check the active subscriptions
consumer.subscription()

The `KafkaConsumer` class will also offer the possibility to inspect the topics (for instance in terms of the number of partitions), but **not** to modify them. 

We can inspect how many partitions the specific topic is made of:

In [ ]:
# print the list of partition IDs 
# e.g. a topic with tree partitions will have partition IDs {0, 1, 2}
consumer.partitions_for_topic('my_awesome_topic')

We can instruct the consumer to `poll` (i.e., ask for all new messages stored in the topic) with a given cadence/logic.

For instance, we can set the consumer to read only 10 messages at a time, with a timeout between subsequent readouts of a given $\Delta t$.

In [ ]:
# set up the polling strategy for the consumer
consumer.poll(timeout_ms=0,         # do not enable dead-times before one poll to the next
              max_records=None,     # do not limit the number of records to consume at once 
              update_offsets=True   # update the reading offsets on this topic
             )

Now the consumer is ready to poll messages (until it is stopped or it reaches a timeout).

Let's look for all messages in the consumer:

In [ ]:
# this consumer will keep polling for messages 
# until stopped by the user
# (or reaches the consumer_timeout_ms, if specified)
for message in consumer:
    print (message)

The reading offset can also be reset to the beginning of the topic, allowing for re-reading the entire topic:

In [ ]:
# go back to the beginning of the topic
consumer.seek_to_beginning()

# read the entire topic and print it out
for message in consumer:
    print (message)

The message content (`ConsumerRecord`) can be quite messy, but it can be easily inspected by parsing only the relevant information.

In [ ]:
# import packages to interpret dates
from datetime import datetime

# go back to the beginning of the topic
consumer.seek_to_beginning()

# break down the message into its main components
for message in consumer:
    print ("{%d}:%d [%s] k=%s v=%s" % (message.partition,
                          message.offset,
                          datetime.fromtimestamp(message.timestamp/1000).time(),
                          message.key,
                          message.value))

Let's change the topic to which the consumer is subscribed and make it a partitioned one:

In [ ]:
# subscribe to a partitioned topic
consumer.subscribe('a_partitioned_topic')
consumer.subscription()

By inspecting the number of partitions for this topic, we can now see that there are 2 partitions: partition #0 and partition #1.

In [ ]:
# check the partitions in the partitioned topic
consumer.partitions_for_topic('a_partitioned_topic')

When reading from a partitioned topic, it's easy to observe that the messages are sent to the two partitions in a seemingly arbitrary way.

In [ ]:
# import json to unpack json strings
import json

# go back to the beginning of the topic
consumer.seek_to_beginning()

# read messages from the beginning of the topic
# decode the json into the python dictionary format
for message in consumer:
    print ("%d:%d:\t v=%s" % (message.partition,
                          message.offset,
                          json.loads(message.value)))

## Creating a consumer accessing only one partition

Publishing records to a partitioned topic is typically transparent to the user: the producer publishes to the topic, and the Kafka cluster will redirect the message to the partition leader, later replicating it to the followers.

The same applies to a generic consumer. As we have just seen, data is consumed from all partitions within the topic.

In some cases, however, it may be more suitable to instantiate multiple consumers, with each one reading from a specific partition of a topic.

Let's create a consumer specifically designed to access the data from partition #0 of the previous partitioned topic.

In [ ]:
# import TopicPartition to be able to assign a partition to a consumer
from kafka import TopicPartition

# create a standard consumer with a timeout of 10 seconds
consumer_part_0 = KafkaConsumer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                                client_id='consumer_n_0',
                                consumer_timeout_ms=10000)

# assign the consumer to a specific topic-partition combination
consumer_part_0.assign([TopicPartition('a_partitioned_topic', # topic
                                       0                      # partition id
                                       )
                        ]) 

In [ ]:
# go back to the beginning of the topic (for this specific partition)
consumer_part_0.seek_to_beginning()

# read messages from the beginning of the topic (for this specific partition)
# decode the json into the python dictionary format
for message in consumer_part_0:
    print ("%d:%d:\t v=%s" % (message.partition,
                          message.offset,
                          json.loads(message.value)))

## Creating a consumer group

Multiple consumers can read from the same topic.

In Kafka, every consumer is part of a consumer group (even a single consumer is part of its own consumer group, really).

A consumer group consists of one or more cooperating consumers that gather data from the same topic. The consumer group dynamically balances the load across its members and redistributes the consume calls.

If a consumer within a consumer group fails, the remaining consumers in the same group will continue to read the entire data from the subscribed topic.

In [ ]:
# create `consumer_one` to read from a specific partition
# assign this consumer to a group
consumer_one = KafkaConsumer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
                             client_id='consumer_one',
                             group_id='my_group',                       # the same group will be used by all consumers
                             auto_offset_reset='earliest',
                             consumer_timeout_ms=10000)

In [ ]:
from kafka import ConsumerRebalanceListener

# subscribe `consumer_one` to the partitioned topic
consumer_one.subscribe('a_partitioned_topic',
                       listener=ConsumerRebalanceListener()             # set a logic on how the group should be re-balance 
                                                                        # when partitions are added or removed
                                                                        # or when new consumers are assigned to partitions
                       )

---

### Consuming messages as a group
Use the `ConsumerGroup` notebook to:
1. create a second consumer `consumer_two`
2. assign it to the same consumer group `my_group`
3. subscribe to the same topic `a_partitioned_topic`

Each consumer within a group is going to be an independent process (should be run in parallel from the others) and will provide access to a fraction of the incoming data

In [ ]:
# check the partitions assigned to `consumer_one`
consumer_one.assignment()

Start the two consumers and process data in parallel from the two partitions.

Typically, one would achieve this by running the two consumers in separate threads, processes, or executors, depending on the desired computing architecture.

In [ ]:
# use multiple consumers in parallel (`consumer_one` in this notebook)
for message in consumer_one:
    print ("%d:%d: k=%s v=%s" % (message.partition,
                          message.offset,
                          message.key,
                          json.loads(message.value)))

---

## Reading from the Kafka+Spark results topic

Let's subscribe to the `results` topic and monitor the frauds

In [ ]:
consumer.subscribe('results')

for message in consumer:
    print ("%d:%d: k=%s v=%s" % (message.partition,
                          message.offset,
                          message.key,
                          message.value))
    print ('      --> sending alert message to user {}\n'.format(message.key.decode('ascii')))